# Benchmarking causalml learners
The aim of this notebook is to compare the performance of the causal learners implemented in the [causalml](https://github.com/uber/causalml) Python package with that of the original R implementations.

First, we setup a port to the R implementations using `rpy2`. We then write a simple wrapper function for rboost and rlasso, which can be passed to the evaluation API of *JustCause*. We aim to compare the meta learner with the same base-learners in both cases, which is why we use xgboost and glmnet to provide the same base learners used in the original R implementation.

## Details of R implementations
### Underlying Base Learners
The details of the R implementations used below can be found in the respective files [here](https://github.com/xnie/rlearner/tree/master/R).
From our analysis, RLasso uses an ElasticNet with the lasso objective thus we should set `l1_ratio=1`. (See [sklearn docu](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)). But in the case below we use the `glmnet` package based on the same Fortran library as the original R code used by the rlearner implementation. 


The details of the RBoost are best replicated by using the xgboost python package, as the authors use the same package in the R implementation. 

### Propensity Scores 
The propensity scores in our R-Learner wrapper are calculated



In [2]:
import numpy as np

from justcause.learners.utils import install_r_packages
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages

from rpy2.robjects.vectors import FloatVector, IntVector, StrVector

# activate numpy conversion
numpy2ri.activate()

install_r_packages(['devtools'])

def install_rlearner():
    
    d = {'package.dependencies': 'package_dot_dependencies',
             'package_dependencies': 'package_uscore_dependencies'}
    
    if not rpackages.isinstalled('rlearner'):
        custom_install = importr('devtools',robject_translations = d)
        custom_install.install_github("xnie/rlearner")
    
    return importr('rlearner', robject_translations = d)



def rlearner_lasso(train, test):
    rlearner = install_rlearner()
    train_X, train_t, train_y = train.np.X, train.np.t, train.np.y
    test_X, test_t, test_y = test.np.X, test.np.t, test.np.y
    
    
    model = rlearner.rlasso(train_X, IntVector(train_t), FloatVector(train_y))
    
    return (
        np.array(ro.r.predict(model, train_X)).reshape(1, -1)[0], 
        np.array(ro.r.predict(model, test_X)).reshape(1, -1)[0]
           )

def rlearner_boost(train, test):
    rlearner = install_rlearner()
    train_X, train_t, train_y = train.np.X, train.np.t, train.np.y
    test_X, test_t, test_y = test.np.X, test.np.t, test.np.y
    
    
    model = rlearner.rboost(train_X, IntVector(train_t), FloatVector(train_y))
    
    return (
        np.array(ro.r.predict(model, train_X)).reshape(1, -1)[0], 
        np.array(ro.r.predict(model, test_X)).reshape(1, -1)[0]
           )


/Users/MaximilianFranz/anaconda3/envs/justcause/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/MaximilianFranz/anaconda3/envs/justcause/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.
/Users/MaximilianFranz/anaconda3/envs/justcause/lib/python3.7/site

In [3]:
from sklearn.model_selection import train_test_split

from justcause.data.sets import load_ihdp
from justcause.evaluation import evaluate_ite
from justcause.learners import RLearner
from justcause.metrics import pehe_score

## Minimal Working Example

In [4]:
rep = load_ihdp(select_rep=[0])[0] # select first replication only 
train, test = train_test_split(rep, train_size=0.8)
train_ite, test_ite = rlearner_boost(train, test)

In [7]:
rep = load_ihdp(select_rep=[0])[0] # select first replication only 
train, test = train_test_split(rep, train_size=0.8)
train_ite, test_ite = rlearner_lasso(train, test)

# Evaluation
## R-Learner Lasso : Python vs R 

In [9]:
from glmnet import ElasticNet
from sklearn.linear_model import ElasticNet

methods = [rlearner_lasso, RLearner(ElasticNet(l1_ratio=1))]
replications = load_ihdp(select_rep=np.arange(100))
result = evaluate_ite(replications, methods, metrics=pehe_score)

TypeError: fit() got an unexpected keyword argument 'sample_weight'

In [5]:
result

,method,train,pehe_score-mean,pehe_score-median,pehe_score-std
0,rlearner_lasso,True,6.445494,3.373901,8.140446
1,rlearner_lasso,False,6.445494,3.373901,8.140446
2,"RLearner(outcome=ElasticNet, effect=ElasticNet)",True,4.128427,2.230457,5.136343
3,"RLearner(outcome=ElasticNet, effect=ElasticNet)",False,4.128427,2.230457,5.136343


## R-Learner Boost : Python vs R 

In [6]:
from xgboost import XGBRegressor

methods = [rlearner_boost, RLearner(XGBRegressor())]

result = evaluate_ite(, 100, methods, metrics=pehe_score)

[10:44:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:44:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [7]:
result 

,method,train,pehe_score-mean,pehe_score-median,pehe_score-std
0,rlearner_boost,True,2.036739,0.939122,2.913037
1,rlearner_boost,False,2.036739,0.939122,2.913037
2,"RLearner(outcome=XGBRegressor, effect=XGBRegre...",True,2.822892,1.373032,4.210576
3,"RLearner(outcome=XGBRegressor, effect=XGBRegre...",False,2.822892,1.373032,4.210576
